# HYBRID (LSTM -> Transformer)


##01. Imports and Configuration

This block sets up libraries, deterministic behaviour and immutable hyperparameters.
Why: ensures run-to-run reproducibility and identical outputs.
Method choices: fixed seeds; TF32 disabled; America/New_York 16:00 close; constant hyperparameters.

In [ ]:
import os, sys, json, math, time, hashlib, zipfile, random, shutil
from pathlib import Path
from typing import List, Dict, Tuple, Optional
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

# ------------------------- Determinism prelude -------------------------
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
if hasattr(torch.backends, "cuda") and hasattr(torch.backends.cuda, "matmul"):
    torch.backends.cuda.matmul.allow_tf32 = False
if hasattr(torch.backends, "cudnn"):
    torch.backends.cudnn.allow_tf32 = False
try:
    torch.use_deterministic_algorithms(True)
except Exception:
    pass

def set_seed(seed: int = SEED):
    import random as _r, numpy as _np, torch as _t
    _r.seed(seed); _np.random.seed(seed)
    _t.manual_seed(seed)
    if _t.cuda.is_available():
        _t.cuda.manual_seed(seed); _t.cuda.manual_seed_all(seed)

# ----------------------------- Config -----------------------------
MODEL_ID = "HYBRID"
TICKERS = ["AAPL","AMZN","MSFT","TSLA","AMD"]

TRAIN_START = "2021-02-03"
TRAIN_END   = "2022-12-30"
VAL_START   = "2023-01-03"
VAL_END     = "2023-05-31"
TEST_START  = "2023-06-01"
TEST_END    = "2023-12-28"
TEST_LEN    = 146

LOOKBACK = 90
HORIZON  = 1

BATCH_SIZE      = 64
EPOCHS_BASE     = 120
REFIT_MONTHLY   = True
REFIT_EPOCHS    = 6
LR              = 1e-3
WEIGHT_DECAY    = 1e-4
PATIENCE        = 15
GRAD_CLIP       = 1.0

NUM_WORKERS         = 0
PIN_MEMORY          = torch.cuda.is_available()
PERSISTENT_WORKERS  = False

DA_EPS = 0.0010
TIMEZONE = "America/New_York"
MARKET_CUTOFF = "16:00"
MEAN_BIAS_TAU = 10.0

REFERENCE_FEATURES_JSON = "transformer_outputs/TRANSFORMER/AAPL/run_config_TRANSFORMER_AAPL.json"

PRICE_COLS = ["Open","High","Low","Close","Volume"]
TECH_COLS  = ["Return_1d","LogRet_1d","Vol_7","Vol_21","SMA_7","SMA_21","RSI_14","MACD","MACD_Signal"]

MODULE_ROOT = Path("outputs")
MODEL_DIR   = MODULE_ROOT / MODEL_ID
TPL_DIR     = MODULE_ROOT / "third_party_licenses"
for d in [MODULE_ROOT, MODEL_DIR, TPL_DIR]:
    d.mkdir(parents=True, exist_ok=True)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
set_seed(SEED)

# --------------------- Sentiment detection (prefix-driven) ---------------------
SENTIMENT_PREFIXES = ("Tw_", "Rd_", "Nw_SP500_")

def is_sentiment_feature(col: str) -> bool:
    return isinstance(col, str) and col.startswith(SENTIMENT_PREFIXES)

def derive_sentiment_cols(features: List[str]) -> List[str]:
    return [c for c in features if is_sentiment_feature(c)]

# --------------------- Feature-order -------------------------
def load_reference_features(path: str) -> Optional[List[str]]:
    p = Path(path)
    if not p.exists():
        return None
    try:
        ref = json.loads(p.read_text())
        if "features_used" in ref and isinstance(ref["features_used"], list):
            feats = list(ref["features_used"])
        else:
            feats = list(ref.get("features", {}).get("list", []))
        return feats
    except Exception:
        return None

def features_sha256(features: List[str]) -> str:
    payload = json.dumps({"features_used": features}, sort_keys=True, separators=(",", ":")).encode("utf-8")
    h = hashlib.sha256(); h.update(payload); return h.hexdigest()

def build_features(df: pd.DataFrame) -> List[str]:
    ref_feats = load_reference_features(REFERENCE_FEATURES_JSON)

    def local_fallback() -> List[str]:
        core = PRICE_COLS + TECH_COLS
        def pick(pref): return sorted([c for c in df.columns if c.startswith(pref)])
        return [c for c in core if c in df.columns] + pick("Tw_") + pick("Rd_") + pick("Nw_SP500_")

    if ref_feats:
        feats = [c for c in ref_feats if c in df.columns]
        fb = local_fallback()
        # Strict list equality to guarantee identical order; include SHA guidance for audits.
        if feats != fb:
            raise AssertionError(
                "Features parity violation: reference features_used differ in ORDER or MEMBERSHIP from local derivation.\n"
                f"ref_len={len(feats)} fb_len={len(fb)}\n"
                f"ref_sha={features_sha256(feats)}\nfb_sha={features_sha256(fb)}\n"
                "Regenerate the reference pack (LSTM(SE)/Transformer) or align upstream CSV schemas to restore identical features_used."
            )
    else:
        feats = local_fallback()

    for rc in PRICE_COLS:
        assert rc in feats, f"Required column missing: {rc}"
    return feats

## Data Loading

This block defines file discovery and enforced temporal splits.
Why: exact reading of final CSVs and fixed Train/Validation/Test ranges.
Method choices: fixed splits 2021-02-03→2022-12-30 (Train), 2023-01-03→2023-05-31 (Validation), 2023-06-01→2023-12-28 (Test, n=146).

In [ ]:
# ------------------------ I/O & Sanitisation -----------------------
def load_input_csv(ticker: str) -> pd.DataFrame:
    p = Path(f"{ticker}_input.csv")
    if not p.exists():
        raise FileNotFoundError(f"Expected {p.name} next to the script.")
    df = pd.read_csv(p)
    need = ["date","ticker","Target","Close"]
    miss = sorted(list(set(need) - set(df.columns)))
    if miss:
        raise AssertionError(f"{p.name} missing columns: {miss}")
    df["date"] = pd.to_datetime(df["date"])
    df = df.sort_values("date").reset_index(drop=True)
    return df

def sanitise_frame(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for c in df.columns:
        if c not in ["date","ticker"]:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    # sentiment zero-encoding (robust to missing *_count columns)
    for fam, cnt in [("Tw_", "Tw_count"), ("Rd_", "Rd_count"), ("Nw_SP500_", "Nw_SP500_count")]:
        fam_cols = [c for c in df.columns if c.startswith(fam)]
        if fam_cols:
            df[fam_cols] = df[fam_cols].fillna(0.0)
            cnt_series = df[cnt] if cnt in df.columns else pd.Series(0, index=df.index, dtype=float)
            cnt_series = pd.to_numeric(cnt_series, errors="coerce").fillna(0.0)
            mask0 = (cnt_series == 0.0)
            df.loc[mask0, fam_cols] = 0.0
    for cnt in ["Tw_count","Rd_count","Nw_SP500_count"]:
        if cnt in df: df[cnt] = pd.to_numeric(df[cnt], errors="coerce").fillna(0.0)

    # prices/technicals: causal ffill
    for c in PRICE_COLS + TECH_COLS:
        if c in df.columns:
            df[c] = df[c].ffill()

    # Target AFTER features exist
    if "Close" not in df.columns:
        raise AssertionError("Close column missing for Target computation.")
    df["Target"] = df["Close"].shift(-1)

    # fill remaining NaNs (features only)
    feat_cols = [c for c in df.columns if c not in ["date","ticker","Target"]]
    df[feat_cols] = df[feat_cols].fillna(0.0)
    return df

def split_data(df: pd.DataFrame):
    tr = df[(df["date"] >= pd.to_datetime(TRAIN_START)) & (df["date"] <= pd.to_datetime(TRAIN_END))].copy().reset_index(drop=True)
    va = df[(df["date"] >= pd.to_datetime(VAL_START))   & (df["date"] <= pd.to_datetime(VAL_END))].copy().reset_index(drop=True)
    te = df[(df["date"] >= pd.to_datetime(TEST_START))  & (df["date"] <= pd.to_datetime(TEST_END))].copy().reset_index(drop=True)
    if len(te) != TEST_LEN:
        raise AssertionError(f"Test rows={len(te)} != {TEST_LEN}.")
    for name, d in [("Train",tr),("Val",va),("Test",te)]:
        assert d["date"].is_monotonic_increasing, f"{name} not sorted"
        assert d["date"].is_unique, f"{name} duplicate dates"
    return tr, va, te

def soft_target_check(df: pd.DataFrame, name: str):
    delta = (df["Target"] - df["Close"].shift(-1)).abs()
    mism = int((delta > 1e-6).sum())
    if mism > 0:
        print(f"[warn] {name}: {mism} rows where Target != Close.shift(-1) (tol 1e-6).")

##Preprocessing

This block applies train-only scaling and windowing.
Why: standardise features while preserving zeros for Tw_/Rd_/Nw_SP500_* and prepare sequential windows.
Method choices: Zero-preserving standardiser; no shuffling; Validation transformed using the history-fitted scaler during refits to avoid leakage.

In [ ]:
# ------------------ Zero-preserving z-scores ----------------
class ZeroPreservingStandardScaler:
    def __init__(self, sentiment_cols: List[str], eps: float = 1e-8):
        self.sentiment = set([c for c in sentiment_cols if isinstance(c, str)])
        self.mu, self.sigma = {}, {}
        self.feature_order = None
        self.eps = float(eps)
    def fit(self, X_df: pd.DataFrame):
        X = X_df.astype(np.float32)
        self.feature_order = list(X.columns)
        for c in self.feature_order:
            col = X[c].to_numpy(np.float32)
            if c in self.sentiment:
                nz = col != 0.0
                vals = col[nz]
                mu = float(vals.mean()) if vals.size else 0.0
                sd = float(vals.std(ddof=0)) if vals.size else 1.0
                sd = max(sd, self.eps)
            else:
                mu = float(col.mean()); sd = max(float(col.std(ddof=0)), self.eps)
            self.mu[c], self.sigma[c] = mu, sd
        return self
    def transform(self, X_df: pd.DataFrame) -> pd.DataFrame:
        X = X_df.astype(np.float32).copy()
        for c in self.feature_order:
            if c in self.sentiment:
                nz = X[c].to_numpy(np.float32) != 0.0
                X.loc[nz, c] = (X.loc[nz, c] - self.mu[c]) / self.sigma[c]
                X.loc[~nz, c] = 0.0
            else:
                X[c] = (X[c] - self.mu[c]) / self.sigma[c]
        X = X.replace([np.inf, -np.inf], 0.0).fillna(0.0)
        return X

def audit_zero_preservation(df_original: pd.DataFrame, df_scaled: pd.DataFrame, sentiment_cols: List[str]):
    bad = []
    for c in sentiment_cols:
        if c not in df_scaled.columns or c not in df_original.columns:
            continue
        mask = df_original[c] == 0
        if mask.any() and not np.allclose(df_scaled.loc[mask, c].to_numpy(), 0.0):
            bad.append(c)
    if bad:
        raise AssertionError(f"Zero preservation failed for: {bad}")

def zero_preservation_report(df_before: pd.DataFrame, df_after: pd.DataFrame, sentiment_cols: List[str]) -> pd.DataFrame:
    rows = []
    for c in sentiment_cols:
        if c not in df_before.columns or c not in df_after.columns:
            continue
        xin = df_before[c].to_numpy()
        xout = df_after[c].to_numpy()
        violations = int(((xin == 0) & (xout != 0)).sum())
        rows.append({"feature": c, "violations": violations, "ok": violations == 0})
    return pd.DataFrame(rows)

# ------------------ Windowing (residual-anchored) -------------------
def make_windows_from_slice(df_slice: pd.DataFrame, feature_cols: List[str], lookback: int = LOOKBACK, horizon: int = HORIZON) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    dfw = df_slice.sort_values("date").reset_index(drop=True).copy()
    A = dfw[feature_cols].to_numpy(np.float32)
    C = dfw["Close"].to_numpy(np.float32)
    X_list, y_list, c_list = [], [], []
    N = len(dfw)
    for i in range(lookback, N - horizon + 1):
        Xi = A[i-lookback:i, :]
        yi_idx = i - 1 + horizon
        if yi_idx < 0 or yi_idx >= len(C): continue
        yi = C[yi_idx]
        ci = C[i-1]
        if np.isfinite(Xi).all() and np.isfinite(yi) and np.isfinite(ci):
            X_list.append(Xi); y_list.append(yi); c_list.append(ci)
    if not X_list:
        return np.empty((0, lookback, len(feature_cols)), np.float32), np.empty((0,1), np.float32), np.empty((0,1), np.float32)
    X = np.stack(X_list).astype(np.float32)
    y = np.array(y_list, dtype=np.float32).reshape(-1,1)
    c = np.array(c_list, dtype=np.float32).reshape(-1,1)
    return X, y, c

def make_val_windows_with_carryin(trXy: pd.DataFrame, vaXy: pd.DataFrame, feature_cols: List[str], lookback: int = LOOKBACK, horizon: int = HORIZON) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    if lookback <= 0 or horizon <= 0:
        return np.empty((0,0,0), np.float32), np.empty((0,1), np.float32), np.empty((0,1), np.float32)
    ctx = trXy.tail(lookback).copy()
    cat = pd.concat([ctx, vaXy], ignore_index=True)
    A = cat[feature_cols].to_numpy(np.float32)
    C = cat["Close"].to_numpy(np.float32)
    X_list, y_list, c_list = [], [], []
    n_val = len(vaXy); Ncat = len(cat)
    for j in range(n_val):
        k = lookback + j
        start = k - lookback; end = k
        label_idx = (k - 1) + horizon
        if start < 0 or label_idx >= Ncat: continue
        Xi = A[start:end, :]; yi = float(C[label_idx]); ci = float(C[end-1])
        if Xi.shape[0] == lookback and np.isfinite(Xi).all() and math.isfinite(yi) and math.isfinite(ci):
            X_list.append(Xi); y_list.append(yi); c_list.append(ci)
    if not X_list:
        return np.empty((0, lookback, A.shape[1] if A.ndim == 2 else 0), np.float32), np.empty((0,1), np.float32), np.empty((0,1), np.float32)
    X = np.stack(X_list).astype(np.float32)
    y = np.array(y_list, dtype=np.float32).reshape(-1,1)
    c = np.array(c_list, dtype=np.float32).reshape(-1,1)
    return X, y, c

class TimeSeriesDataset(Dataset):
    def __init__(self, X, y, c):
        self.X = X.astype(np.float32); self.y = y.astype(np.float32); self.c = c.astype(np.float32)
    def __len__(self): return len(self.X)
    def __getitem__(self, i):
        return torch.from_numpy(self.X[i]), torch.from_numpy(self.y[i]), torch.from_numpy(self.c[i])

def _worker_init_fn(worker_id):
    seed = SEED + worker_id + 1
    random.seed(seed); np.random.seed(seed)

def make_loader(X, y, c, batch=BATCH_SIZE):
    # Note: shuffle=False for sequence continuity; no generator needed.
    return DataLoader(
        TimeSeriesDataset(X, y, c),
        batch_size=batch,
        shuffle=False,
        drop_last=False,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        persistent_workers=PERSISTENT_WORKERS if NUM_WORKERS > 0 else False,
        worker_init_fn=_worker_init_fn if NUM_WORKERS > 0 else None,
    )

##Model Definition

This block defines the Hybrid LSTM → Transformer model and scheduler helper.
Why: exact architecture and forward pass for residual-anchored level forecasts.
Method choices: sinusoidal positional encoding; TransformerEncoder; MLP head produces delta then add last Close.

In [ ]:
# ----------------------------- Model -------------------------------
class SinusoidalPositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=1000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)
        div = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(pos * div); pe[:, 1::2] = torch.cos(pos * div)
        self.register_buffer("pe", pe.unsqueeze(0), persistent=False)
    def forward(self, x): return x + self.pe[:, :x.size(1), :]

class HybridLSTMTransformer(nn.Module):
    def __init__(self, input_dim, lstm_hidden=128, lstm_layers=1, d_model=128, nhead=4, enc_layers=2, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, lstm_hidden, lstm_layers, batch_first=True, bidirectional=False)
        self.lstm_drop = nn.Dropout(dropout)
        self.proj_in = nn.Linear(lstm_hidden, d_model)
        self.posenc  = SinusoidalPositionalEncoding(d_model)
        enc_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=4*d_model,
                                               dropout=dropout, batch_first=True, activation="relu")
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=enc_layers)
        self.enc_norm = nn.LayerNorm(d_model)
        self.head = nn.Sequential(nn.Linear(lstm_hidden + d_model, 256), nn.ReLU(), nn.Dropout(dropout), nn.Linear(256, 1))
    def forward(self, x):
        x = torch.nan_to_num(x)
        lstm_out, _ = self.lstm(x)
        lstm_last = self.lstm_drop(lstm_out[:, -1, :])
        z = self.proj_in(lstm_out)
        z = self.posenc(z); z = self.encoder(z); z = self.enc_norm(z)
        enc_pool = z.mean(dim=1)
        delta = self.head(torch.cat([lstm_last, enc_pool], dim=1))
        return torch.nan_to_num(delta)

def _make_plateau_scheduler(opt):
    try: return ReduceLROnPlateau(opt, mode="min", factor=0.5, patience=3, verbose=False)
    except TypeError: return ReduceLROnPlateau(opt, mode="min", factor=0.5, patience=3)

##Training

This block trains the base model and supports ES with ReduceLROnPlateau.
Why: obtain a fitted model for the initial expanding-origin phase.
Method choices: accumulate losses on device; early stopping on Validation.

In [ ]:
# --------------------------- Training loop -------------------------
def train_model(model, tr_loader, va_loader, epochs=EPOCHS_BASE, lr=LR, wd=WEIGHT_DECAY,
                patience=PATIENCE, grad_clip=GRAD_CLIP):
    model.to(DEVICE)
    loss_fn = nn.MSELoss()
    opt = AdamW(model.parameters(), lr=lr, weight_decay=wd)
    sched = _make_plateau_scheduler(opt)

    best = float("inf"); stall = 0; best_state = None; best_epoch = 0
    tr_hist, va_hist = [], []
    ep_done = 0

    for ep in range(epochs):
        ep_done = ep + 1

        # Accumulate losses on device; convert once per epoch to avoid host syncs each batch.
        tr_loss_sum_t = torch.zeros((), device=DEVICE)
        tr_count = 0

        model.train()
        for xb, yb, cb in tr_loader:
            xb, yb, cb = xb.to(DEVICE, non_blocking=True), yb.to(DEVICE, non_blocking=True), cb.to(DEVICE, non_blocking=True)
            opt.zero_grad(set_to_none=True)
            delta = model(xb)
            pred_level = cb + delta
            loss = loss_fn(pred_level, yb)
            if not torch.isfinite(loss):
                continue
            loss.backward()
            if grad_clip is not None:
                nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            opt.step()
            # accumulate weighted by batch size
            tr_loss_sum_t += loss.detach() * xb.size(0)
            tr_count += xb.size(0)

        tr_loss_epoch = (tr_loss_sum_t / max(tr_count, 1)).item() if tr_count else float("inf")
        tr_hist.append(tr_loss_epoch)

        va_loss_sum_t = torch.zeros((), device=DEVICE)
        va_count = 0
        model.eval()
        with torch.no_grad():
            for xb, yb, cb in va_loader:
                xb, yb, cb = xb.to(DEVICE, non_blocking=True), yb.to(DEVICE, non_blocking=True), cb.to(DEVICE, non_blocking=True)
                delta = model(xb)
                pred_level = cb + delta
                l = loss_fn(pred_level, yb)
                if not torch.isfinite(l):
                    continue
                va_loss_sum_t += l * xb.size(0)
                va_count += xb.size(0)

        va_loss = (va_loss_sum_t / max(va_count, 1)).item() if va_count else float("inf")
        va_hist.append(va_loss)
        sched.step(va_loss if np.isfinite(va_loss) else best)

        if va_loss < best - 1e-9:
            best = va_loss; stall = 0; best_epoch = int(ep) + 1
            best_state = {k: v.detach().cpu() for k, v in model.state_dict().items()}
        else:
            stall += 1
            if stall >= patience:
                break

    if best_state is not None:
        model.load_state_dict({k: v.to(DEVICE) for k, v in best_state.items()})
    return model, tr_hist, va_hist, best_epoch, ep_done

##Evaluation

This block computes deterministic metrics and the trading diagnostic.
Why: level accuracy plus rule-based long/short with ε-threshold sign.
Method choices: RMSE/MAE; Theil’s U2 vs naive last Validation close; Directional Accuracy ε=0.0010; Sharpe/MaxDD at 0 and 10 bps.

In [ ]:
# ------------------------------ Metrics ----------------------------
def rmse(a, b): a = np.asarray(a, float); b = np.asarray(b, float); return float(np.sqrt(np.mean((a-b)**2)))
def mae(a, b):  a = np.asarray(a, float); b = np.asarray(b, float); return float(np.mean(np.abs(a-b)))
def theils_u2(pred, actual, naive): return rmse(pred, actual) / max(rmse(naive, actual), 1e-12)

def directional_accuracy_eps_from_levels(pred_levels, actual_levels, eps=DA_EPS, prev0=None):
    pred = np.asarray(pred_levels, float); act = np.asarray(actual_levels, float)
    base_prev = act[0] if prev0 is None else float(prev0)
    prev = np.concatenate([[base_prev], act[:-1]])
    ret_pred = (pred - prev) / np.where(prev == 0.0, 1.0, prev)
    ret_act  = (act  - prev) / np.where(prev == 0.0, 1.0, prev)
    mask = np.abs(ret_act) > eps
    n = int(mask.sum())
    if n == 0: return float("nan"), 0.0, 0
    da  = float(np.mean(np.sign(ret_pred[mask]) == np.sign(ret_act[mask])))
    cov = float(n / len(ret_act))
    return da, cov, n

def backtest_long_short_series(pred_levels, actual_levels, eps=DA_EPS, cost_bps=0.0, prev0=None):
    pred = np.asarray(pred_levels, float); act = np.asarray(actual_levels, float)
    base_prev = act[0] if prev0 is None else float(prev0)
    prev = np.concatenate([[base_prev], act[:-1]])
    ret_act  = (act  - prev) / np.where(prev == 0.0, 1.0, prev)
    ret_pred = (pred - prev) / np.where(prev == 0.0, 1.0, prev)
    pos = np.where(ret_pred >  eps,  1, np.where(ret_pred < -eps, -1, 0)).astype(int)
    pos_prev = np.concatenate([[0], pos[:-1]])
    cost = np.abs(pos - pos_prev) * (cost_bps/10000.0)
    strat = pos_prev * ret_act - cost
    eq = np.cumprod(1.0 + strat); peak = np.maximum.accumulate(eq)
    dd = 1.0 - eq/np.maximum(peak, 1e-12)
    sharpe = float((np.mean(strat) / (np.std(strat) + 1e-12)) * np.sqrt(252.0))
    return {"sharpe": sharpe, "maxdd": float(dd.max()), "turnover": int((pos != pos_prev).sum()),
            "ret_pred_min": float(ret_pred.min()), "ret_pred_max": float(ret_pred.max())}

##Outputs and Artefacts

This block handles provenance metadata, plots, run_config building, monthly refit evaluation, writing artefacts, packaging, and patch/verify.
Why: meet finalisation gate requirements and produce auditable deliverables.
Method choices: parameter count; zero-preservation spot CSV; environment manifest; file hashes; ZIP pack; parity stub.

In [ ]:
# -------------- Provenance templates and param count ---------------
TEMPLATE_COMMITS = {
    "transformer_template": {
        "repo_url": "https://github.com/oliverguhr/transformer-time-series-prediction",
        "license": "MIT",
        "commit_sha": "4e120f3f9b67482dd9a8ca88b49671881120d20f"
    },
    "lstm_template": {
        "repo_url": "https://github.com/jinglescode/time-series-forecasting-pytorch",
        "license": "Apache-2.0",
        "commit_sha": "749689e352290616e69d9b3e243af36936328964"
    }
}
def hybrid_param_count(input_dim, lstm_hidden=128, lstm_layers=1, d_model=128, nhead=4, enc_layers=2):
    def lstm_layer_params(in_dim, hid): return 4*(in_dim*hid + hid*hid + hid)
    total = lstm_layer_params(input_dim, lstm_hidden)
    for _ in range(1, lstm_layers): total += lstm_layer_params(lstm_hidden, lstm_hidden)
    total += lstm_hidden*d_model + d_model
    D = d_model
    per_enc = (3*D*D + 3*D) + (D*D + D) + (4*D*D + 4*D) + (4*D*D + D) + (4*D)
    total += enc_layers * per_enc
    total += (lstm_hidden + d_model)*256 + 256 + 256 + 1
    return int(total)

# ------------------------------ Plots ------------------------------
def plot_training_curves(out_dir: Path, ticker: str, tr_hist, va_hist):
    plt.figure(); plt.plot(tr_hist, label="train_loss"); plt.plot(va_hist, label="val_loss")
    plt.title(f"{ticker}: training curves"); plt.xlabel("epoch"); plt.ylabel("loss"); plt.legend(); plt.tight_layout()
    p = out_dir / f"training_curves_{MODEL_ID}_{ticker}.png"; plt.savefig(p); plt.close(); return p

def plot_actual_vs_pred(out_dir: Path, ticker: str, dates, actual, pred):
    fig, ax = plt.subplots(figsize=(9,4))
    ax.plot(dates, actual, label="Actual"); ax.plot(dates, pred, label="Predicted")
    ax.set_title(f"{ticker}: Actual vs Predicted"); ax.set_xlabel("date"); ax.set_ylabel("price level"); ax.legend()
    fig.tight_layout(); p = out_dir / f"actual_vs_pred_{MODEL_ID}_{ticker}.png"; fig.savefig(p); plt.close(fig); return p

def plot_residuals_strip(out_dir: Path, ticker: str, dates, residual):
    fig, ax = plt.subplots(figsize=(9,3))
    ax.plot(dates, residual); ax.axhline(0, ls="--")
    ax.set_title(f"{ticker}: Residuals"); ax.set_xlabel("date"); ax.set_ylabel("actual - pred")
    fig.tight_layout(); p = out_dir / f"residuals_strip_{MODEL_ID}_{ticker}.png"; fig.savefig(p); plt.close(fig); return p

# ---------------------------- run_config ---------------------------
def build_run_config_spec(ticker, features, scaler_summary, refit_dates: List[str], param_count: int,
                          per_refit_seeds: List[int], base_best_epoch: int, refit_epochs_achieved: List[int]):
    env = {"python": sys.version.split()[0], "numpy": np.__version__, "pandas": pd.__version__,
           "torch": torch.__version__, "device": DEVICE, "timezone": TIMEZONE, "price_timestamp": MARKET_CUTOFF}
    spec = {
        "pipeline": "hybrid_v3_residual_anchor",
        "model_id": MODEL_ID,
        "ticker": ticker,
        "splits": {"train": [TRAIN_START, TRAIN_END], "val": [VAL_START, VAL_END], "test": [TEST_START, TEST_END]},
        "split_test": {"start": TEST_START, "end": TEST_END, "n": TEST_LEN},
        "features": {"list": features, "order_fixed": True, "count": len(features),
                     "schema": {"target": "Target", "close_raw": "Close"}, "use_sentiment": True},
        "scaler": {"type": "ZeroPreservingStandardScaler", "zero_preserved_families": list(SENTIMENT_PREFIXES),
                   "summary": scaler_summary, "scope": "FINAL_REFIT"},
        "model": {"templates": TEMPLATE_COMMITS,
                  "fusion": "LSTM -> Linear -> PosEnc -> TransformerEncoder; concat(LSTM_last, mean_pool(Transformer)) -> MLP; output delta; add last Close",
                  "hparams": {"input_dim": len(features), "lstm_hidden": 128, "lstm_layers": 1,
                              "d_model": 128, "nhead": 4, "enc_layers": 2, "dropout": 0.2},
                  "parameter_count": param_count},
        "training": {"optimizer": "AdamW", "lr": LR, "weight_decay": WEIGHT_DECAY,
                     "scheduler": {"name": "ReduceLROnPlateau", "factor": 0.5, "patience": 3},
                     "loss": "MSE(level)", "early_stopping": {"on":"val_loss","patience": PATIENCE},
                     "epochs_base": EPOCHS_BASE, "refit_monthly": REFIT_MONTHLY, "epochs_per_refit": REFIT_EPOCHS,
                     "epochs_per_refit_achieved": refit_epochs_achieved, "base_best_epoch": base_best_epoch,
                     "batch_size": BATCH_SIZE, "seed": SEED, "grad_clip": GRAD_CLIP},
        "dataloader": {"num_workers": NUM_WORKERS, "pin_memory": PIN_MEMORY, "persistent_workers": PERSISTENT_WORKERS},
        "cadence": {"lookback": LOOKBACK, "horizon": HORIZON, "mode": "monthly_refit", "refit_dates": refit_dates},
        "cadence_str": "monthly_refit",
        "window_L": LOOKBACK,
        "metrics_policy": {"theils_u2": "vs_naive_last_close",
                           "da_epsilon": {"eps": DA_EPS, "strict_gt": True},
                           "trading_rule": "eps-threshold sign", "cost_bps": [0,10]},
        "environment": env,
        "provenance": {
            "origin": "Adapted",
            "repos": [
                {"url": TEMPLATE_COMMITS["transformer_template"]["repo_url"],
                 "commit_sha": TEMPLATE_COMMITS["transformer_template"]["commit_sha"],
                 "license": TEMPLATE_COMMITS["transformer_template"]["license"]},
                {"url": TEMPLATE_COMMITS["lstm_template"]["repo_url"],
                 "commit_sha": TEMPLATE_COMMITS["lstm_template"]["commit_sha"],
                 "license": TEMPLATE_COMMITS["lstm_template"]["license"]}
            ],
            "attribution": "Adapted; training/evaluation wrappers, residual anchor, monthly refit, and artefact schema authored."
        },
        "features_used": features,
        "seeds": {"global": SEED, "per_refit": per_refit_seeds},
        "parameter_count": param_count,
        "batch_size": BATCH_SIZE,
        "epochs_per_refit": REFIT_EPOCHS,
        "epochs_per_refit_achieved": refit_epochs_achieved,
        "refit_dates": refit_dates,
        "clock": f"{TIMEZONE}@{MARKET_CUTOFF}",
        "target_definition": "Close.shift(-1) after features",
        "y_scaled": False
    }
    canonical = json.dumps(spec, sort_keys=True, separators=(",", ":"))
    spec["protocol_hash"] = hashlib.sha256(canonical.encode("utf-8")).hexdigest()
    return spec

# ------------- Train once, then monthly refit for Test --------------
def expanding_origin_predict_monthly(model_cfg: Dict,
                                     df_train: pd.DataFrame, df_val: pd.DataFrame, df_test: pd.DataFrame,
                                     features: List[str]):
    sentiment_cols = derive_sentiment_cols(features)

    train_scaler = ZeroPreservingStandardScaler(sentiment_cols).fit(df_train[features])
    trS = train_scaler.transform(df_train[features])
    vaS = train_scaler.transform(df_val[features])
    audit_zero_preservation(df_train, trS, sentiment_cols)
    audit_zero_preservation(df_val,   vaS, sentiment_cols)

    def assemble(base_df, scaled_df):
        out = scaled_df.copy()
        out.insert(0, "date", base_df["date"].values)
        out["Close"]  = base_df["Close"].values
        out["Target"] = base_df["Target"].values
        return out

    trXy, vaXy = assemble(df_train, trS), assemble(df_val, vaS)
    Xtr, ytr, ctr = make_windows_from_slice(trXy, features, LOOKBACK, HORIZON)
    Xva, yva, cva = make_val_windows_with_carryin(trXy, vaXy, features, LOOKBACK, HORIZON)
    if len(Xtr)==0 or len(Xva)==0:
        raise AssertionError(f"Empty Train/Val windows: Xtr={len(Xtr)}, Xva={len(Xva)}")

    set_seed(SEED)
    model = HybridLSTMTransformer(input_dim=len(features), **model_cfg).to(DEVICE)
    tr_loader = make_loader(Xtr, ytr, ctr, BATCH_SIZE)
    va_loader = make_loader(Xva, yva, cva, BATCH_SIZE)
    model, tr_hist, va_hist, best_epoch, _ep_done = train_model(model, tr_loader, va_loader, epochs=EPOCHS_BASE)

    preds, actuals, dates = [], [], []
    refit_dates: List[str] = []
    per_refit_seeds: List[int] = []
    refit_epochs_achieved: List[int] = []
    base_hist = pd.concat([df_train, df_val], ignore_index=True)

    current_scaler = train_scaler
    cur_month = None
    for i in range(len(df_test)):
        dt = df_test.iloc[i]["date"]
        ym = (int(dt.year), int(dt.month))

        if REFIT_MONTHLY and (cur_month is None or ym != cur_month):
            cur_month = ym
            refit_dates.append(pd.Timestamp(dt).strftime("%Y-%m-%d"))
            per_refit_seeds.append(SEED)

            hist = pd.concat([base_hist, df_test.iloc[:i]], ignore_index=True)

            # Fit scaler on history up to prediction date; then transform *Validation* with that same scaler.
            # This preserves causality and avoids leakage from future Test data into scaling statistics.
            current_scaler = ZeroPreservingStandardScaler(sentiment_cols).fit(hist[features])
            histS = current_scaler.transform(hist[features])
            audit_zero_preservation(hist, histS, sentiment_cols)

            histXy = assemble(hist, histS)
            Xh, yh, ch = make_windows_from_slice(histXy, features, LOOKBACK, HORIZON)
            if len(Xh) > 0:
                set_seed(SEED)
                model = HybridLSTMTransformer(input_dim=len(features), **model_cfg).to(DEVICE)
                h_loader = make_loader(Xh, yh, ch, BATCH_SIZE)

                vaS_refit = current_scaler.transform(df_val[features])
                audit_zero_preservation(df_val, vaS_refit, sentiment_cols)
                vaXy_refit = assemble(df_val, vaS_refit)
                Xva_refit, yva_refit, cva_refit = make_val_windows_with_carryin(histXy, vaXy_refit, features, LOOKBACK, HORIZON)
                va_loader_refit = make_loader(Xva_refit, yva_refit, cva_refit, BATCH_SIZE)
                model, _, _, _best_refit, ep_done_refit = train_model(model, h_loader, va_loader_refit, epochs=REFIT_EPOCHS, patience=2)
                refit_epochs_achieved.append(int(ep_done_refit))
            else:
                refit_epochs_achieved.append(0)

        hist = pd.concat([base_hist, df_test.iloc[:i]], ignore_index=True)
        lastL = hist[features].tail(LOOKBACK)
        if lastL.shape[0] < LOOKBACK:
            raise AssertionError("Insufficient history for lookback window.")
        lastL_scaled = current_scaler.transform(lastL).to_numpy(np.float32)
        last_close = float(hist["Close"].iloc[-1])
        xb = torch.from_numpy(np.nan_to_num(lastL_scaled, nan=0.0, posinf=0.0, neginf=0.0)[None, ...]).to(DEVICE)
        with torch.no_grad():
            delta_hat = float(torch.nan_to_num(model(xb)).cpu().numpy().reshape(-1)[0])
        yhat = last_close + delta_hat
        preds.append(yhat)
        actuals.append(float(df_test.iloc[i]["Close"]))
        dates.append(pd.Timestamp(dt))

    dfp = pd.DataFrame({"date": dates, "y_true": actuals, "y_hat": preds})
    assert len(dfp) == TEST_LEN and dfp["date"].min().strftime("%Y-%m-%d")==TEST_START and dfp["date"].max().strftime("%Y-%m-%d")==TEST_END
    return dfp, tr_hist, va_hist, best_epoch, refit_dates, per_refit_seeds, current_scaler, vaS, sentiment_cols, refit_epochs_achieved

# ---------------- Zero-encoding audit artefact ---------------------
def write_zero_encoding_check(out_dir: Path, val_raw: pd.DataFrame, val_scaled: pd.DataFrame, sentiment_cols: List[str]):
    cols = [c for c in sentiment_cols if c in val_raw.columns and c in val_scaled.columns]
    if not cols:
        return
    dfm = val_raw[["date"] + cols].merge(
        val_scaled[["date"] + cols], on="date", suffixes=("_raw","_scaled")
    )
    # Pick the first month present for a quick spot check; auditors can adjust window if needed.
    if not dfm.empty:
        first_month = (dfm["date"].dt.to_period("M").iloc[0])
        mask = dfm["date"].dt.to_period("M") == first_month
        df_sample = dfm[mask].head(8)
    else:
        df_sample = dfm.head(8)
    df_sample.to_csv((out_dir / "zero_encoding_check_val_sample.csv"), index=False)

# --------------- Write per-ticker artefacts ------------------------
def write_scaler_summary_csv(out_dir: Path,
                             scaler: ZeroPreservingStandardScaler,
                             val_raw_for_check: pd.DataFrame,
                             val_scaled_for_check: pd.DataFrame,
                             sentiment_cols: List[str]):
    rows = []
    zrep = zero_preservation_report(val_raw_for_check, val_scaled_for_check, sentiment_cols)
    zmap = {r["feature"]: (int(r["violations"]), bool(r["ok"])) for _, r in zrep.iterrows()}
    for c in scaler.mu.keys():
        vio, ok = zmap.get(c, (0, True)) if c in sentiment_cols else (0, True)
        rows.append({
            "feature": c,
            "mu": float(scaler.mu[c]),
            "sigma": float(scaler.sigma[c]),
            "is_sentiment": (c in scaler.sentiment),
            "violations": int(vio),
            "ok": bool(ok),
            "scope": "FINAL_REFIT"
        })
    pd.DataFrame(rows).to_csv(out_dir / "scaler_summary_FINAL_REFIT.csv", index=False)

def write_ticker_artefacts(ticker: str,
                           dfpred: pd.DataFrame,
                           tr_hist, va_hist, best_epoch: int,
                           refit_dates: List[str], per_refit_seeds: List[int],
                           refit_epochs_achieved: List[int],
                           scaler: ZeroPreservingStandardScaler,
                           features_used: List[str],
                           full_df: pd.DataFrame,
                           valS: pd.DataFrame,
                           sentiment_cols: List[str]):
    out_dir = MODEL_DIR / ticker
    out_dir.mkdir(parents=True, exist_ok=True)

    dates  = pd.to_datetime(dfpred["date"]).to_list()
    actual = dfpred["y_true"].to_numpy()
    pred   = dfpred["y_hat"].to_numpy()
    residual = actual - pred

    mean_gap = float(np.abs(np.mean(pred) - np.mean(actual)))
    assert mean_gap < MEAN_BIAS_TAU, f"Level sanity failed: |mean(y_hat) - mean(y_true)|={mean_gap:.3f} >= τ={MEAN_BIAS_TAU}."

    pd.DataFrame({
        "date": dfpred["date"],
        "y_true": actual,
        "y_hat": pred,
        "residual": residual,
        "in_sample_flag": 0
    }).to_csv(out_dir / f"predictions_{MODEL_ID}_{ticker}.csv", index=False)

    _, va, _ = split_data(full_df)
    val_last_close = float(va["Close"].iloc[-1])
    naive = np.concatenate([[val_last_close], actual[:-1]])

    da_value, cov_da, n_da = directional_accuracy_eps_from_levels(pred, actual, eps=DA_EPS, prev0=val_last_close)
    u2 = theils_u2(pred, actual, naive)
    summ0  = backtest_long_short_series(pred, actual, eps=DA_EPS, cost_bps=0.0, prev0=val_last_close)
    summ10 = backtest_long_short_series(pred, actual, eps=DA_EPS, cost_bps=10.0, prev0=val_last_close)

    diag = {
        "eps": DA_EPS,
        "turnover": summ0["turnover"],
        "pred_ret_min": summ0["ret_pred_min"],
        "pred_ret_max": summ0["ret_pred_max"],
        "note": "If turnover==0, Sharpe/MaxDD=0 is expected by construction."
    }
    (out_dir / f"trading_diag_{MODEL_ID}_{ticker}.json").write_text(json.dumps(diag, indent=2), encoding="utf-8")

    def _zeroish(x): return abs(x) < 1e-12 or (isinstance(x, float) and (math.isnan(x) or math.isinf(x)))
    if summ0["turnover"] > 0 and (_zeroish(summ0["sharpe"]) and _zeroish(summ0["maxdd"])):
        raise AssertionError(f"{ticker}: zero trading metrics at 0 bps despite turnover>0.")
    if summ10["turnover"] > 0 and (_zeroish(summ10["sharpe"]) and _zeroish(summ10["maxdd"])):
        raise AssertionError(f"{ticker}: zero trading metrics at 10 bps despite turnover>0.")

    metrics = {
        "RMSE": rmse(pred, actual),
        "MAE": mae(pred, actual),
        "U2": u2,
        "DA_epsilon": da_value,
        "Coverage": cov_da,
        "n": int(len(actual)),
        "Sharpe_0bps":  float(summ0["sharpe"]),
        "MaxDD_0bps":   float(summ0["maxdd"]),
        "Sharpe_10bps": float(summ10["sharpe"]),
        "MaxDD_10bps":  float(summ10["maxdd"]),
        "best_epoch": int(best_epoch)
    }
    (out_dir / f"metrics_{MODEL_ID}_{ticker}.json").write_text(json.dumps(metrics, indent=2), encoding="utf-8")

    plot_training_curves(out_dir, ticker, tr_hist, va_hist)
    plot_actual_vs_pred(out_dir, ticker, dates, actual, pred)
    plot_residuals_strip(out_dir, ticker, dates, residual)

    # Build run_config with enriched clock/target plus scaler summary (including ok/violations)
    val_raw_slice = full_df[(full_df["date"]>=VAL_START) & (full_df["date"]<=VAL_END)][["date"] + [c for c in features_used if c in full_df.columns]].reset_index(drop=True)
    val_scaled_for_check = pd.concat([val_raw_slice[["date"]], valS[[c for c in features_used if c in valS.columns]].reset_index(drop=True)], axis=1)
    scaler_summary = {c: {"mu": float(scaler.mu[c]), "sigma": float(scaler.sigma[c]),
                          "is_sentiment": c in scaler.sentiment} for c in scaler.mu.keys()}
    zrep = zero_preservation_report(val_raw_slice, val_scaled_for_check, sentiment_cols)
    scaler_summary["zero_preservation_report"] = zrep.to_dict(orient="records")

    param_count = hybrid_param_count(len(features_used))
    run_cfg = build_run_config_spec(
        ticker=ticker, features=features_used, scaler_summary=scaler_summary,
        refit_dates=refit_dates, param_count=param_count, per_refit_seeds=per_refit_seeds,
        base_best_epoch=int(metrics["best_epoch"]), refit_epochs_achieved=[int(x) for x in refit_epochs_achieved]
    )
    (out_dir / f"run_config_{MODEL_ID}_{ticker}.json").write_text(json.dumps(run_cfg, indent=2), encoding="utf-8")

    cad_lines = [f"{i+1:02d}: {d}  (first trading day of month segment)" for i, d in enumerate(refit_dates)]
    (out_dir / f"cadence_{ticker}.txt").write_text("\n".join(cad_lines) + ("\n" if cad_lines else ""), encoding="utf-8")

    # Zero-preservation compliance CSV with ok/violations
    write_zero_encoding_check(out_dir, val_raw_slice, val_scaled_for_check, sentiment_cols)
    write_scaler_summary_csv(out_dir, scaler, val_raw_slice, val_scaled_for_check, sentiment_cols)

    print(f"[{ticker}] RMSE={metrics['RMSE']:.4f}  U2={metrics['U2']:.4f}  "
          f"DA={metrics['DA_epsilon']:.4f} cov={metrics['Coverage']:.4f} "
          f"sh0={metrics['Sharpe_0bps']:.4f} dd0={metrics['MaxDD_0bps']:.4f} "
          f"sh10={metrics['Sharpe_10bps']:.4f} dd10={metrics['MaxDD_10bps']:.4f}")

    return {
        "ticker": ticker,
        "RMSE": metrics["RMSE"], "MAE": metrics["MAE"], "U2": metrics["U2"],
        "DA_epsilon": metrics["DA_epsilon"], "Coverage": metrics["Coverage"], "n": metrics["n"],
        "Sharpe_0bps": metrics["Sharpe_0bps"], "Sharpe_10bps": metrics["Sharpe_10bps"],
        "MaxDD_0bps": metrics["MaxDD_0bps"], "MaxDD_10bps": metrics["MaxDD_10bps"],
        "parameter_count": param_count
    }

# ----------- Root provenance, licences, packaging, hashes ----------
MIT_TEXT = """MIT License

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to do so, subject to the
following conditions:
The above copyright notice and this permission notice shall be included
in all copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.
"""

APACHE2_TEXT = """Apache License
Version 2.0, January 2004
https://www.apache.org/licenses/LICENSE-2.0.txt
"""

def write_third_party_and_provenance():
    TPL_DIR.mkdir(parents=True, exist_ok=True)
    readme = [
        "Third-party licences",
        "",
        "This folder mirrors licences for third-party repositories adapted or referenced by the HYBRID runner.",
        f"- Transformer template: {TEMPLATE_COMMITS['transformer_template']['repo_url']} ({TEMPLATE_COMMITS['transformer_template']['license']}), commit {TEMPLATE_COMMITS['transformer_template']['commit_sha']}",
        f"- LSTM template: {TEMPLATE_COMMITS['lstm_template']['repo_url']} ({TEMPLATE_COMMITS['lstm_template']['license']}), commit {TEMPLATE_COMMITS['lstm_template']['commit_sha']}",
        "",
        "Attribution: HYBRID adapts ideas from the above; date-split loader, zero-preserving scaler, residual anchor, and monthly refit evaluator are authored here."
    ]
    (TPL_DIR / "README.md").write_text("\n".join(readme), encoding="utf-8")
    (TPL_DIR / "LICENSE_MIT.txt").write_text(MIT_TEXT, encoding="utf-8")
    (TPL_DIR / "LICENSE_Apache-2.0.txt").write_text(APACHE2_TEXT, encoding="utf-8")

    prov_rows = []
    for name, obj in TEMPLATE_COMMITS.items():
        prov_rows.append({
            "component": name,
            "repo_url": obj["repo_url"],
            "license": obj["license"],
            "commit_sha": obj["commit_sha"],
            "where_used": "HYBRID model reference; encoder design"
        })
    pd.DataFrame(prov_rows).to_csv(MODEL_DIR / "code_provenance.csv", index=False)

def write_model_readme():
    readme = [
        f"{MODEL_ID} outputs",
        "",
        "Splits",
        f"- Train: {TRAIN_START} -> {TRAIN_END}",
        f"- Val:   {VAL_START} -> {VAL_END}",
        f"- Test:  {TEST_START} -> {TEST_END} (n=146)",
        "",
        f"Timezone: {TIMEZONE}, price timestamp: {MARKET_CUTOFF} close.",
        "",
        "Metrics policy",
        "- Theil's U2 vs naive last close (last Validation close as carry-in).",
        "- Directional Accuracy on returns with eps=0.0010; 'coverage' = fraction of Test days with |actual return| > eps.",
        "- Trading rule: epsilon-threshold sign (positions are flat when |predicted next-day return| <= eps); turnover can be low in calm periods."
    ]
    (MODULE_ROOT / "README.md").write_text("\n".join(readme), encoding="utf-8")

def write_env_manifest():
    lines = [f"Python {sys.version.split()[0]}",
             f"Pandas {pd.__version__}",
             f"NumPy {np.__version__}",
             f"Torch {torch.__version__}",
             f"Device {DEVICE}",
             f"NUM_WORKERS {NUM_WORKERS}",
             f"PIN_MEMORY {PIN_MEMORY}",
             f"PERSISTENT_WORKERS {PERSISTENT_WORKERS}",
             f"PYTHONHASHSEED {os.environ.get('PYTHONHASHSEED','')}",
             f"CUBLAS_WORKSPACE_CONFIG {os.environ.get('CUBLAS_WORKSPACE_CONFIG','')}"]
    if torch.cuda.is_available():
        try:
            cap = torch.cuda.get_device_capability(); lines.append(f"CUDA capability {cap}")
        except Exception:
            pass
    (MODULE_ROOT / "env_manifest.txt").write_text("\n".join(lines) + "\n", encoding="utf-8")

def write_cross_ticker_table(rows):
    cols = ["ticker","RMSE","MAE","U2","DA_epsilon","Coverage","n",
            "Sharpe_0bps","Sharpe_10bps","MaxDD_0bps","MaxDD_10bps","parameter_count"]
    pd.DataFrame(rows)[cols].to_csv(MODULE_ROOT / "cross_ticker_table_HYBRID.csv", index=False)

def write_features_parity_stub(any_features: List[str]):
    stub = {
        "HYBRID": features_sha256(any_features),
        "note": "Compare this sha256 against LSTM_SE and TRANSFORMER packs for cross-model features parity."
    }
    (MODULE_ROOT / "cross_model_features_parity_stub.json").write_text(json.dumps(stub, indent=2), encoding="utf-8")

def write_file_hashes():
    def sha256_of(path: Path):
        h = hashlib.sha256()
        with open(path, "rb") as f:
            for chunk in iter(lambda: f.read(1<<20), b""):
                h.update(chunk)
        return h.hexdigest()
    recs = []
    for p in MODULE_ROOT.rglob("*"):
        if p.is_file() and not (p.parent == MODULE_ROOT / "hybrid" and p.name == "hybrid.zip"):
            recs.append({"path": str(p.relative_to(MODULE_ROOT)),
                         "size": p.stat().st_size,
                         "sha256": sha256_of(p),
                         "mtime_utc": time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime(p.stat().st_mtime))})
    (MODULE_ROOT / "file_hashes.json").write_text(json.dumps(recs, indent=2), encoding="utf-8")

def package_zip():
    assert (MODULE_ROOT / "env_manifest.txt").exists(), "Missing env_manifest.txt"
    assert (MODULE_ROOT / "file_hashes.json").exists(), "Missing file_hashes.json"
    assert (MODULE_ROOT / "cross_ticker_table_HYBRID.csv").exists(), "Missing cross_ticker_table_HYBRID.csv"
    assert (TPL_DIR / "README.md").exists(), "Missing third_party_licenses/README.md"
    assert (TPL_DIR / "LICENSE_MIT.txt").exists(), "Missing LICENSE_MIT.txt"
    assert (TPL_DIR / "LICENSE_Apache-2.0.txt").exists(), "Missing LICENSE_Apache-2.0.txt"

    outdir = MODULE_ROOT / "hybrid"
    outdir.mkdir(parents=True, exist_ok=True)
    zpath = outdir / "hybrid.zip"
    if zpath.exists(): zpath.unlink()

    with zipfile.ZipFile(zpath, "w", zipfile.ZIP_DEFLATED) as z:
        z.write(MODULE_ROOT / "env_manifest.txt", arcname="env_manifest.txt")
        z.write(MODULE_ROOT / "file_hashes.json", arcname="file_hashes.json")
        z.write(MODULE_ROOT / "cross_ticker_table_HYBRID.csv", arcname="cross_ticker_table_HYBRID.csv")
        for p in TPL_DIR.rglob("*"):
            if p.is_file():
                z.write(p, arcname=str(p.relative_to(MODULE_ROOT)))
        for p in MODEL_DIR.rglob("*"):
            if p.is_file():
                z.write(p, arcname=str(p.relative_to(MODULE_ROOT)))
        stub = MODULE_ROOT / "cross_model_features_parity_stub.json"
        if stub.exists(): z.write(stub, arcname="cross_model_features_parity_stub.json")
        readme = MODULE_ROOT / "README.md"
        if readme.exists(): z.write(readme, arcname="README.md")
    print(f"[ZIP] wrote {zpath.resolve()}")
    return zpath

# --------------- Patch + Verify (idempotent) -----------------------
def patch_verify_refresh():
    rows = []
    for t in TICKERS:
        base = MODEL_DIR / t
        pred_path = base / f"predictions_{MODEL_ID}_{t}.csv"
        met_path  = base / f"metrics_{MODEL_ID}_{t}.json"
        rc_path   = base / f"run_config_{MODEL_ID}_{t}.json"
        assert pred_path.exists() and met_path.exists() and rc_path.exists(), f"Missing artefacts for {t}"
        dfp = pd.read_csv(pred_path, parse_dates=["date"])
        full_df = load_input_csv(t)
        _, va_slice, _ = split_data(full_df)
        prev0 = float(va_slice["Close"].iloc[-1])

        y = dfp["y_true"].to_numpy(float); p = dfp["y_hat"].to_numpy(float)
        naive = np.concatenate([[prev0], y[:-1]])
        da, cov, _n_da = directional_accuracy_eps_from_levels(p, y, eps=DA_EPS, prev0=prev0)
        u2 = theils_u2(p, y, naive)
        m0 = backtest_long_short_series(p, y, eps=DA_EPS, cost_bps=0.0, prev0=prev0)
        m10 = backtest_long_short_series(p, y, eps=DA_EPS, cost_bps=10.0, prev0=prev0)

        prev_metrics = {}
        try:
            prev_metrics = json.loads(met_path.read_text())
        except Exception:
            prev_metrics = {}

        new_metrics = {
            "RMSE": rmse(p, y),
            "MAE": mae(p, y),
            "U2": u2,
            "DA_epsilon": da,
            "Coverage": cov,
            "n": int(len(y)),
            "Sharpe_0bps": float(m0["sharpe"]), "MaxDD_0bps": float(m0["maxdd"]),
            "Sharpe_10bps": float(m10["sharpe"]), "MaxDD_10bps": float(m10["maxdd"])
        }
        # Carry best_epoch forward for continuity (optional per checklist).
        if "best_epoch" in prev_metrics:
            new_metrics["best_epoch"] = int(prev_metrics["best_epoch"])

        met_path.write_text(json.dumps(new_metrics, indent=2), encoding="utf-8")

        rc = json.loads(rc_path.read_text())
        rc.setdefault("window_L", LOOKBACK)
        rc.setdefault("cadence_str", "monthly_refit")
        rc.setdefault("split_test", {"start": TEST_START, "end": TEST_END, "n": TEST_LEN})
        rc.setdefault("dataloader", {"num_workers": NUM_WORKERS, "pin_memory": PIN_MEMORY, "persistent_workers": PERSISTENT_WORKERS})
        rc.setdefault("clock", f"{TIMEZONE}@{MARKET_CUTOFF}")
        rc.setdefault("target_definition", "Close.shift(-1) after features")
        rc.setdefault("y_scaled", False)
        rc_path.write_text(json.dumps(rc, indent=2), encoding="utf-8")

        rows.append({"ticker": t, "parameter_count": rc.get("parameter_count"), **new_metrics})

        if m0["turnover"] > 0 and abs(new_metrics.get("Sharpe_0bps", 0.0)) < 1e-12 and abs(new_metrics.get("MaxDD_0bps", 0.0)) < 1e-12:
            raise AssertionError(f"[verify] {t} 0bps zero metrics despite turnover>0.")
        if m10["turnover"] > 0 and abs(new_metrics.get("Sharpe_10bps", 0.0)) < 1e-12 and abs(new_metrics.get("MaxDD_10bps", 0.0)) < 1e-12:
            raise AssertionError(f"[verify] {t} 10bps zero metrics despite turnover>0.")

    write_cross_ticker_table(rows)
    write_file_hashes()
    print("[patch] metrics schema enforced, trading recomputed, cross-ticker + hashes refreshed.")

# -------------------------- Download helper ------------------------
def try_download(path: Path):
    try:
        from google.colab import files
        files.download(str(path))
    except Exception as e:
        print(f"[download] Skipped ({e})")

# ------------------------------- Driver ----------------------------
def run_ticker(ticker: str):
    print(f"\n=== {ticker} ===")
    df_raw = load_input_csv(ticker)
    df = sanitise_frame(df_raw)
    feat_cols = build_features(df)
    tr, va, te = split_data(df)
    soft_target_check(tr, f"{ticker} Train")
    soft_target_check(va, f"{ticker} Val")
    soft_target_check(te, f"{ticker} Test")

    model_cfg = dict(lstm_hidden=128, lstm_layers=1, d_model=128, nhead=4, enc_layers=2, dropout=0.2)
    dfpred, tr_hist, va_hist, best_epoch, refit_dates, per_refit_seeds, final_scaler, vaS_scaled, sentiment_cols, refit_epochs_achieved = expanding_origin_predict_monthly(
        model_cfg=model_cfg, df_train=tr, df_val=va, df_test=te, features=feat_cols
    )

    row = write_ticker_artefacts(
        ticker=ticker,
        dfpred=dfpred,
        tr_hist=tr_hist,
        va_hist=va_hist,
        best_epoch=best_epoch,
        refit_dates=refit_dates,
        per_refit_seeds=per_refit_seeds,
        refit_epochs_achieved=refit_epochs_achieved,
        scaler=final_scaler,
        features_used=feat_cols,
        full_df=df,
        valS=pd.concat([va[["date"]].reset_index(drop=True),
                        final_scaler.transform(va[feat_cols]).reset_index(drop=True)], axis=1),
        sentiment_cols=sentiment_cols
    )
    return row, feat_cols

def main():
    if (MODULE_ROOT / "hybrid").exists():
        shutil.rmtree(MODULE_ROOT / "hybrid", ignore_errors=True)

    rows = []
    last_feats = None
    for t in TICKERS:
        row, feats = run_ticker(t)
        rows.append(row)
        last_feats = feats

    write_third_party_and_provenance()
    write_model_readme()
    write_cross_ticker_table(rows)
    write_env_manifest()

    if last_feats:
        write_features_parity_stub(last_feats)

    patch_verify_refresh()
    write_file_hashes()
    zpath = package_zip()
    try_download(zpath)
    print("\nAll done. Root:", MODULE_ROOT)

if __name__ == "__main__":
    main()


=== AAPL ===
[AAPL] RMSE=2.1435  U2=1.0066  DA=0.5328 cov=0.9384 sh0=-1.1925 dd0=0.1043 sh10=-1.2656 dd10=0.1061

=== AMZN ===
[AMZN] RMSE=2.5214  U2=1.0051  DA=0.4627 cov=0.9178 sh0=-0.7389 dd0=0.1661 sh10=-0.8021 dd10=0.1728

=== MSFT ===
[MSFT] RMSE=4.6477  U2=0.9988  DA=0.5775 cov=0.9726 sh0=0.0000 dd0=0.0000 sh10=0.0000 dd10=0.0000

=== TSLA ===
[TSLA] RMSE=7.5103  U2=1.0026  DA=0.4789 cov=0.9726 sh0=-1.3263 dd0=0.3002 sh10=-1.3699 dd10=0.3030

=== AMD ===
[AMD] RMSE=3.0082  U2=1.0021  DA=0.4786 cov=0.9589 sh0=0.0000 dd0=0.0000 sh10=0.0000 dd10=0.0000
[patch] metrics schema enforced, trading recomputed, cross-ticker + hashes refreshed.
[ZIP] wrote /content/outputs/hybrid/hybrid.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


All done. Root: outputs
